### Loading Libraries

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

### Loading Data

In [2]:
survey_year = '2018'
survey_type = 'Climate Change'
cities_df = pd.read_csv('Cities/Cities Responses/'+survey_year+'_Full_Cities_Dataset.csv')
cities_info_df = pd.read_csv('Cities/Cities Disclosing/'+survey_year+'_Cities_Disclosing_to_CDP.csv')
corps_cc_df = pd.read_csv('Corporations/Corporations Responses/'+survey_type+'/'+survey_year+'_Full_Climate_Change_Dataset.csv')
na_hq_df = pd.read_csv('Supplementary Data/Locations of Corporations/NA_HQ_public_data.csv')
corps_cc_info_df = pd.read_csv('Corporations/Corporations Disclosing/'+survey_type+'/'+survey_year+'_Corporates_Disclosing_to_CDP_Climate_Change.csv')
census_df = pd.read_csv('Supplementary Data/CDC 500 Cities Census Tract Data/500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2019_release.csv')
social_df = pd.read_csv('Supplementary Data/CDC Social Vulnerability Index 2018/SVI2018_US.csv')
social_county_df = pd.read_csv('Supplementary Data/CDC Social Vulnerability Index 2018/SVI2018_US_COUNTY.csv')
uscities_df = pd.read_csv('Supplementary Data/Simple Maps US Cities Data/uscities.csv')

### Cleaning City Data

In [3]:
# cleaning city names
city_names = cities_info_df[['Account Number','City']]
city_names[city_names['City'] == 'Santiago']
city_names.loc[city_names['Account Number'] == 60393, 'City'] = 'Santiago Government'

/Users/Jenny/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


<span style="color:red">To do: Look at other year surveys</span>

In [4]:
# still cleaning city names
cities_clean = pd.merge(cities_df,city_names,how='left',on='Account Number')
cities_clean['Organization'] = cities_clean['City']
cities_clean.drop(labels='City', axis="columns", inplace=True)
cities_clean = cities_clean.rename(columns={'Organization':'City'})
cities_clean.head()

,Questionnaire,Year Reported to CDP,Account Number,City,Country,CDP Region,Parent Section,Section,Question Number,Question Name,Column Number,Column Name,Row Number,Row Name,Response Answer,Comments,File Name,Last update
0,CDP Cities 2018,2018,3203,Chicago,United States of America,North America,Water,Wastewater,16.1,Please provide the percentage breakdown of the...,1,Percentage of wastewater collected,5,Other type of wastewater,NaN,NaN,NaN,24/06/2020 05:28:18 AM
1,CDP Cities 2018,2018,50555,Hamilton,Canada,North America,Climate Hazards,Climate Hazards,2.2a,Please list the most significant climate hazar...,5,Probability of hazard,4,NaN,Medium High,NaN,NaN,24/06/2020 05:28:18 AM
2,CDP Cities 2018,2018,50392,Vitória,Brazil,Latin America,Emissions Reduction: City-wide,Emissions Reduction Actions : City-wide,8.4,What actions is your city taking to reduce emi...,1,Emissions reduction project activity,3,NaN,On-site renewable energy generation,NaN,NaN,24/06/2020 05:28:18 AM
3,CDP Cities 2018,2018,36522,Verbania,Italy,Europe,Strategy,Energy,9.0,Please indicate the energy mix of electricity ...,2,Gas,1,Energy consumption percentage,NaN,NaN,NaN,24/06/2020 05:28:18 AM
4,CDP Cities 2018,2018,35873,Medellín,Colombia,Latin America,Climate Hazards,Climate Hazards,2.2a,Please list the most significant climate hazar...,3,Magnitude of impact,6,NaN,Serious,NaN,NaN,24/06/2020 05:28:18 AM


In [5]:
# cleaning lat and long column
cities_info = cities_info_df.copy()
latlong = cities_info['City Location'].str.strip('POINT ()').str.split(' ', expand=True).rename(columns={0:'Longitude', 1:'Latitude'}) 
cities_info = pd.merge(cities_info,latlong,left_index=True,right_index=True)
cities_info

,Year Reported to CDP,Account Number,Organization,City,Country,CDP Region,Reporting Authority,Access,First Time Discloser,Population,Population Year,City Location,Last update,Longitude,Latitude
0,2018,58671,Helsingør Kommune / Elsinore Municipality,Helsingør,Denmark,Europe,CDP Cities,public,No,62686.0,2017,POINT (12.5921 56.0308),2020-06-25T04:52:49.050,12.5921,56.0308
1,2018,59167,Municipalidad de Providencia,Providencia,Chile,Latin America,CDP Cities,public,No,126487.0,2012,POINT (-70.6092 -33.4313),2020-06-25T04:52:49.050,-70.6092,-33.4313
2,2018,50553,City of Fresno,Fresno,United States of America,North America,CDP Cities,public,Yes,527438.0,2017,POINT (-119.773 36.7468),2020-06-25T04:52:49.050,-119.773,36.7468
3,2018,826407,Munícipio de Mirandela,Mirandela,Portugal,Europe,CDP Cities,public,Yes,11852.0,2011,POINT (-7.1753 41.51),2020-06-25T04:52:49.050,-7.1753,41.51
4,2018,60375,Gobiernación del Archipiélago de San Andrés,Archipiélago de San Andrés,Colombia,Latin America,CDP Cities,public,No,120000.0,2017,POINT (-81.6976 12.5794),2020-06-25T04:52:49.050,-81.6976,12.5794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,2018,74631,Lubumbashi,Lubumbashi,Democratic Republic of the Congo,Africa,CDP Cities,public,Yes,5652000.0,2016,POINT (-27 11),2020-06-25T04:52:49.050,-27,11
637,2018,60898,Ayuntamiento de Naucalpan de Juárez,Naucalpan de Juárez,Mexico,Latin America,CDP Cities,public,No,NaN,0,POINT (-99.2453 19.4631),2020-06-25T04:52:49.050,-99.2453,19.4631
638,2018,60638,Walvis Bay Municipality,Walvis Bay,Namibia,Africa,CDP Cities,non public,No,100000.0,2016,POINT (12.5247 -22.9389),2020-06-25T04:52:49.050,12.5247,-22.9389
639,2018,35995,City of Brasília,Brasília,Brazil,Latin America,CDP Cities,non public,No,3013.0,2017,POINT (-47.8822 -15.7942),2020-06-25T04:52:49.050,-47.8822,-15.7942


### Cleaning Corporations Data

In [6]:
# adding headquarter location data
na_hq = na_hq_df.copy()
corps_cc = corps_cc_df.copy()
corps_cc_info = corps_cc_info_df.copy()

na_hq = na_hq[(na_hq['survey_year'] == survey_year) & (na_hq['theme'] == survey_type)]
na_hq = na_hq[['account_number','hq_country','address_city','address_state','address_lat','adress_long']]
na_hq = na_hq.rename(columns={'address_city':'hq_city','address_state':'hq_state','address_lat':'hq_lat','adress_long':'hq_long'})
corps_cc_hq = pd.merge(corps_cc, na_hq, how='left', on='account_number')

# adding industry data
corps_cc_info = corps_cc_info[['account_number','activities','sectors','industries','primary_activity','primary_sector','primary_industry','primary_questionnaire_sector']]
corps_cc_full = pd.merge(corps_cc_hq, corps_cc_info, how='left', on='account_number')
corps_cc_full.head()

/Users/Jenny/opt/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,account_number,organization,survey_year,response_received_date,accounting_period_to,ors_response_id,submission_date,page_name,module_name,question_number,...,hq_state,hq_lat,hq_long,activities,sectors,industries,primary_activity,primary_sector,primary_industry,primary_questionnaire_sector
0,912,Archer Daniels Midland,2018,2018-09-04T00:00:00Z,NaN,13370,2018-10-24T16:36:50Z,C0.2,C0. Introduction,C0.2,...,NaN,NaN,NaN,"Agricultural products wholesale, Banks, Biofu...","Chemicals, Financial services, Food & beverag...","Food, beverage & agriculture, Materials, Reta...",Oilseed processing,Food & beverage processing,"Food, beverage & agriculture","Food, beverage & tobacco"
1,912,Archer Daniels Midland,2018,2018-09-04T00:00:00Z,NaN,13370,2018-10-24T16:36:50Z,C0.4,C0. Introduction,C0.4,...,NaN,NaN,NaN,"Agricultural products wholesale, Banks, Biofu...","Chemicals, Financial services, Food & beverag...","Food, beverage & agriculture, Materials, Reta...",Oilseed processing,Food & beverage processing,"Food, beverage & agriculture","Food, beverage & tobacco"
2,912,Archer Daniels Midland,2018,2018-09-04T00:00:00Z,NaN,13370,2018-10-24T16:36:50Z,C0.2,C0. Introduction,C0.2,...,NaN,NaN,NaN,"Agricultural products wholesale, Banks, Biofu...","Chemicals, Financial services, Food & beverag...","Food, beverage & agriculture, Materials, Reta...",Oilseed processing,Food & beverage processing,"Food, beverage & agriculture","Food, beverage & tobacco"
3,912,Archer Daniels Midland,2018,2018-09-04T00:00:00Z,NaN,13370,2018-10-24T16:36:50Z,C0.3,C0. Introduction,C0.3,...,NaN,NaN,NaN,"Agricultural products wholesale, Banks, Biofu...","Chemicals, Financial services, Food & beverag...","Food, beverage & agriculture, Materials, Reta...",Oilseed processing,Food & beverage processing,"Food, beverage & agriculture","Food, beverage & tobacco"
4,912,Archer Daniels Midland,2018,2018-09-04T00:00:00Z,NaN,13370,2018-10-24T16:36:50Z,C0.5,C0. Introduction,C0.5,...,NaN,NaN,NaN,"Agricultural products wholesale, Banks, Biofu...","Chemicals, Financial services, Food & beverag...","Food, beverage & agriculture, Materials, Reta...",Oilseed processing,Food & beverage processing,"Food, beverage & agriculture","Food, beverage & tobacco"


### Characterstics of Each City

In [7]:
intro_questions = ('0.1','0.3','0.5','0.6','0.7','0.8')

intro = cities_clean[(cities_clean['Question Number'].isin(intro_questions))][['City','Column Name','Response Answer']]
intro = intro.pivot(index='City', columns='Column Name', values='Response Answer')

cities_desc = cities_clean[(cities_clean['Question Number'] == '0.1') & (cities_clean['Column Number'] == 1)].sort_values('Country')[['City','Country','CDP Region']]

cities_desc = pd.merge(cities_desc,intro,how='left',on='City').sort_values('Country')
cities_desc

,City,Country,CDP Region,Administrative boundary,Annual operating budget,Average altitude (m),Average annual temperature (in Celsius),Budget year end,Budget year start,Current population,...,Land area (in square km),Latitude (e.g. 41.25),Leader name,Leader title,Longitude (e.g. -120.9762),Projected population,Projected population year,Source,Total time in office (years),Year of GDP
0,San Isidro (Argentina),Argentina,Latin America,City / Municipality,5998952920,20,18,2018-12-31,2018-01-01,292.878,...,50,34.28,Gustavo Posse,Intendente Municipal,58.21,300000,2020,NaN,20,NaN
1,Ataliva,Argentina,Latin America,Other: Pueblo,24000,89,18.6,2018-12-31,2018-01-01,2500,...,155,31,Fábio Ssnchez,Presidente Comunal,61,3000,2020,NaN,11,NaN
2,Mendoza,Argentina,Latin America,City / Municipality,2200000,NaN,24,2018-12-31,2018-01-01,115041,...,106.07,-32.89084,Rodolfo Alejandro Suárez,Intendente Muncipal,68.82717,119.45,2018,DEIE,4,2015
3,Vicente López,Argentina,Latin America,City / Municipality,4354985.553,11,16.6,2017-12-31,2017-01-01,270929,...,34.45,58.483333,Jorge Macri,Intendente,-34.516667,260000,2018,NaN,8,NaN
4,Rio Grande,Argentina,Latin America,City / Municipality,63335000000,18,6,2017-12-31,2017-01-01,84246,...,12181,-27.18333,Gustavo Melella,Intendente,-67.35,91002,2020,"Dato estimado a partir de datos del: INDEC, Se...",8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,"Dublin, CA",United States of America,North America,City / Municipality,109400000,111,14.7,2019-06-30,2018-07-01,63241,...,15.23,37.7,David Haubert,Mayor,-121.93,75000,2025,Bureau of Economic Analysis (regional data for...,4,2016
484,Ho Chi Minh,Viet Nam,Southeast Asia and Oceania,Special city,NaN,6,26,NaN,NaN,8611000,...,2095,10.75,Nguyen Thanh Phong,Chairman,106.6667,9200000,2020,PSO,NaN,2017
486,Kasama,Zambia,Africa,City / Municipality,34800000,NaN,24,2018-12-31,2018-02-01,350000,...,10000,NaN,Frederick chisanga kabwr,Mayor,NaN,500000,2020,NaN,5,NaN
485,Mazabuka,Zambia,Africa,City / Municipality,24996530,1200,22.3,2019-12-31,2017-01-01,182205,...,2726,NaN,Vincent Lilanda,Mayor,NaN,189593,2020,NaN,5,NaN


In [8]:
#count number of cities by country
cities_desc[['City','Country']].groupby('Country').count().sort_values('City',ascending=False).head(20)

#see which cities are in the census data
cities_desc[cities_info_df['City'].isin(census_df.PlaceName.unique().tolist())][['City','Country']].groupby('Country').count()

#see which cities are in the social vulnerability data -> see lower cell b/c have to match counties with cities first


<ipython-input-8-41166f988200>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cities_desc[cities_info_df['City'].isin(census_df.PlaceName.unique().tolist())][['City','Country']].groupby('Country').count()


,City
Country,
Argentina,1
Australia,2
Brazil,8
Bulgaria,1
Cameroon,1
Canada,1
China,1
Colombia,2
Democratic Republic of the Congo,1


### Cleaning Social Vulnerability Data

In [9]:
# adding cities to social vulnerability dataset

uscities = uscities_df.copy()
social_county = social_county_df.copy()

uscities = uscities.rename(columns = {'county_name':'County','state_id':'State','city':'City'})
uscities = uscities[['City','State','County','lat','lng']].drop_duplicates()
social_county = social_county.rename(columns = {'COUNTY':'County','ST_ABBR':'State'})

social_cities = pd.merge(social_county,uscities,on=['State','County'],how='right')

### Joining States to Cities Dataset

In [10]:
# check if any of the cities in the cities CDP survey dataset are a city whose name is in multiple states
# this would be a problem if we want to add the social vulnerability data to the cities CDP dataset
# given we only have City and Country information in the cities CDP dataset
temp_cities = cities_clean[cities_clean['Country'] == 'United States of America']
city_states = uscities[['City','State','lat','lng']].drop_duplicates()
city_states = city_states[city_states['City'].isin(temp_cities.City.unique().tolist())]
# yup there's definitely a lot of the same cities in different states...need lat long to determine which city survey is referring to                             
city_states.groupby('City').count().head(50)

# fix errors
#'Key West' 24.5551 81.78
#'South Bend' 41.6574 -86.2532
#'Aurora' 41.7606 88.3201
#'Norfolk' 38.8468 -76.2851


cities_info.loc[cities_info['City'] == 'Key West',['Latitude']] = 24.5551
cities_info.loc[cities_info['City'] == 'Key West',['Longitude']] = 81.78

cities_info.loc[cities_info['City'] == 'South Bend',['Latitude']] = 41.6574 
cities_info.loc[cities_info['City'] == 'South Bend',['Longitude']] = -86.2532

cities_info.loc[cities_info['City'] == 'Aurora',['Latitude']] = 41.7606 
cities_info.loc[cities_info['City'] == 'Aurora',['Longitude']] = 88.3201

cities_info.loc[cities_info['City'] == 'Norfolk',['Latitude']] = 38.8468 
cities_info.loc[cities_info['City'] == 'Norfolk',['Longitude']] = -76.2851


# use pythagorean distance to determine most likely state that the city is in
pyth_dist = pd.merge(city_states[['City','State','lat','lng']],cities_info[['City','Latitude','Longitude']],on='City',how='left')
pyth_dist[pyth_dist['City']=='Cleveland']

def pyth(lat,Lat,lng,Long):
    Q = (( float(lat) - abs(float(Lat)) )**2  + (float(lng) + abs(float(Long)))**2 )**0.5
    return Q

pyth_dist['dist'] = pyth_dist.apply(lambda row:pyth(row['lat'],row['Latitude'],row['lng'],row['Longitude']), axis=1)


In [11]:
# finding lat and long that is closest to true value (uscites dataset)
pyth_dist = pyth_dist[pyth_dist['Latitude'].notnull()]
pyth_dist = pyth_dist.loc[pyth_dist.groupby('City')['dist'].idxmin()]
pyth_dist = pyth_dist[pyth_dist['City'].isin(temp_cities.City.unique().tolist())]

pyth_dist.sort_values('dist',ascending=False).head(50)

city_states = pyth_dist[['City','State']]

# add states to cities_clean 
cities_clean = pd.merge(cities_clean,city_states,on='City',how='left')

In [12]:
#see which cities are in the social vulnerability data -> note they should all be in the US but some cities have same name in other countries

cities_desc[cities_desc['City'].isin(social_cities.City.unique().tolist())][['City','Country']].groupby('Country').count()

,City
Country,
Argentina,1
Australia,2
Brazil,2
Canada,9
Chile,1
Colombia,1
Comoros,1
Costa Rica,1
Denmark,1


## Finding Cities Most At Risk

#### Introduction

The relationship between climate change and the most disadvantaged groups in society is well known to be characterized as a **positive feedback loop**. Initial inequality allows the most disadvantaged groups to be impacted disproportionately worse by the impacts of climate change, which results in increased subsequent inequality. 

Finding the cities that are most at risk of this positive feedback loop can be of use to both local and national governments, as it would provide information for what parts of the state and country to direct funding and resources to prevent the cycle from perpetuating. 

In addition, as we have seen with the current pandemic, governments have difficulty providing resources to the corporations that need it most and instead opt for widespread injection of stimulus to corporate balance sheets (**might need link here**). Knowing which companies have locations in the areas that are most disadvantaged (from both a socioeconomic and environmental standpoint), would provide a more direct tool to direct resources.

To find cities that are most at risk from both ends of climate change and inequality, we can define metrics that highlight whether a city is facing social and economic disadvantages in comparison to other cities, across the different types of climate hazards that are likely to prevalent in the future. 


In [15]:
social_cities[['State','County','City','EP_DISABL']][sc_metrics['EP_DISABL'] >= 0]


,State,County,City,EP_DISABL
0,NM,Rio Arriba,Española,16.1
1,NM,Rio Arriba,Dulce,16.1
2,NM,Rio Arriba,La Mesilla,16.1
3,NM,Rio Arriba,Ohkay Owingeh,16.1
4,NM,Rio Arriba,La Villita,16.1
...,...,...,...,...
29191,TX,Jim Hogg,Hebbronville,26.0
29192,TX,Jim Hogg,Thompsonville,26.0
29193,TX,Jim Hogg,Las Lomitas,26.0
29194,TX,Jim Hogg,South Fork Estates,26.0


### Creating Social Vulnerability Metrics

In [74]:
def normalize(df,var):
    min_var = df[var].min()
    max_var = df[var].max()
    diff = max_var - min_var
    df['new_var'] = (df[var] - min_var) / diff
    return df

sc_metrics = social_cities

#poverty metric
sc_pov = sc_metrics[['State','County','City','EP_POV']][sc_metrics['EP_POV'] >= 0]
sc_pov = normalize(sc_pov,'EP_POV').rename(columns={'new_var':'EP_POV_std'})
sc_pov = sc_pov[['City','State','EP_POV_std']].drop_duplicates()

#unemployment metric
sc_unp = sc_metrics[['State','County','City','EP_UNEMP']][sc_metrics['EP_UNEMP'] >= 0]
sc_unp = normalize(sc_unp,'EP_UNEMP').rename(columns={'new_var':'EP_UNEMP_std'})
sc_unp = sc_unp[['City','State','EP_UNEMP_std']].drop_duplicates()

#education metric
sc_edu = sc_metrics[['State','County','City','EP_NOHSDP']][sc_metrics['EP_NOHSDP'] >= 0]
sc_edu = normalize(sc_edu,'EP_NOHSDP').rename(columns={'new_var':'EP_NOHSDP_std'})
sc_edu = sc_edu[['City','State','EP_NOHSDP_std']].drop_duplicates()

#transportation
sc_tsp = sc_metrics[['State','County','City','EP_NOVEH']][sc_metrics['EP_NOVEH'] >= 0]
sc_tsp = normalize(sc_tsp,'EP_NOVEH').rename(columns={'new_var':'EP_NOVEH_std'})
sc_tsp = sc_tsp[['City','State','EP_NOVEH_std']].drop_duplicates()

#proxy for low income (crowded living situation)
sc_crd = sc_metrics[['State','County','City','EP_CROWD']][sc_metrics['EP_CROWD'] >= 0]
sc_crd = normalize(sc_crd,'EP_CROWD').rename(columns={'new_var':'EP_CROWD_std'})
sc_crd = sc_crd[['City','State','EP_CROWD_std']].drop_duplicates()

#proxy for high health costs (disabled but non institutionalized)
sc_dis = sc_metrics[['State','County','City','EP_DISABL']][sc_metrics['EP_DISABL'] >= 0]
sc_dis = normalize(sc_dis,'EP_DISABL').rename(columns={'new_var':'EP_DISABL_std'})
sc_dis = sc_dis[['City','State','EP_DISABL_std']].drop_duplicates()

#proxy for high food costs as % of budget (single parents with young kids)
sc_sng = sc_metrics[['State','County','City','EP_SNGPNT']][sc_metrics['EP_SNGPNT'] >= 0]
sc_sng = normalize(sc_sng,'EP_SNGPNT').rename(columns={'new_var':'EP_SNGPNT_std'})
sc_sng = sc_sng[['City','State','EP_SNGPNT_std']].drop_duplicates()

# disadvantaged score = poverty*unemployment*education*transportation*crowded*disabled*single
disadv_score = pd.merge(sc_pov,sc_unp,on=['State','City'],how='left')
disadv_score = pd.merge(disadv_score,sc_edu,on=['State','City'],how='left')
disadv_score = pd.merge(disadv_score,sc_tsp,on=['State','City'],how='left')
disadv_score = pd.merge(disadv_score,sc_crd,on=['State','City'],how='left')
disadv_score = pd.merge(disadv_score,sc_dis,on=['State','City'],how='left')
disadv_score = pd.merge(disadv_score,sc_sng,on=['State','City'],how='left')
disadv_score['disadv'] = (disadv_score['EP_POV_std']+disadv_score['EP_UNEMP_std']+disadv_score['EP_NOHSDP_std']+disadv_score['EP_NOVEH_std']+disadv_score['EP_CROWD_std']+disadv_score['EP_DISABL_std']+disadv_score['EP_SNGPNT_std'])/7
disadv_score = disadv_score[['City','State','disadv']].drop_duplicates()

disadv_score = disadv_score[disadv_score['disadv'].notnull()]
disadv_score = disadv_score.loc[disadv_score.groupby(['City','State'])['disadv'].idxmin()]

disadv_score[disadv_score['City'].isin(cities_desc.City.unique())].sort_values('disadv',ascending=False)




,City,State,disadv
53621,Roma,TX,0.470701
50592,Manchester,KY,0.417552
52731,Santa Monica,TX,0.407670
50670,Hayward,MO,0.379541
50616,Cleveland,MS,0.377134
...,...,...,...
8680,Clinton,NJ,0.092745
29489,Columbus,ND,0.092120
40901,Hamilton,VA,0.085648
27087,Cologne,MN,0.078885


### Finding Hazards per City

In [76]:
hazards = cities_clean[cities_clean['Question Number'] == '2.2a']
hazards = hazards[hazards['Response Answer'].notnull()]

hazards = hazards.pivot_table(index=['Account Number', 'City', 'State','Row Number'],
                                     columns='Column Name', 
                                     values='Response Answer',
                                     aggfunc=lambda x: ' '.join(x)).reset_index()
hazards

Column Name,Account Number,City,State,Row Number,Anticipated timescale,Climate Hazards,Consequence of hazard,Future change in frequency,Future change in intensity,Hazard status,Impact description,Magnitude of impact,Probability of hazard,Top three assets/ services affected
0,1093,Atlanta,GA,1,Medium-term,Drought,High,Increasing,Increasing,Expected to affect the city in the future,Droughts can affect the supply of potable wate...,Extremely serious,High,Food & agriculture Emergency management Water ...
1,1093,Atlanta,GA,2,Short-term,Heat wave,High,Increasing,Increasing,Expected to affect the city in the future,Around 200 people die in the City of Atlanta a...,Extremely serious,High,Public health Emergency management Energy
2,1093,Atlanta,GA,3,Short-term,Rain storm,High,Increasing,Increasing,Currently affecting the city,Rain storms causes floods in the city overload...,Extremely serious,High,Public health Water supply & sanitation Emerge...
3,1093,Atlanta,GA,4,Long-term,Extreme hot days,Medium,Increasing,Increasing,Expected to affect the city in the future,Extreme hot days overload the electrical syste...,Serious,Medium,Emergency management Energy Public health
4,1093,Atlanta,GA,5,Long-term,Tornado,Low,Increasing,Increasing,Expected to affect the city in the future,Tornadoes have been known to disrupt power lin...,Serious,Low,Energy Emergency management Information & comm...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,74594,Boynton Beach,FL,11,Long-term,Permanent inundation,Medium High,Increasing,Increasing,Expected to affect the city in the future,NaN,Serious,Medium High,NaN
539,74594,Boynton Beach,FL,12,Medium-term,Salt water intrusion,Medium High,Increasing,Increasing,Expected to affect the city in the future,NaN,Serious,Medium High,NaN
540,74594,Boynton Beach,FL,13,Medium-term,Ocean acidification,Do not know,Increasing,Increasing,Expected to affect the city in the future,NaN,Less serious,Do not know,NaN
541,74594,Boynton Beach,FL,14,Medium-term,Vector-borne disease,Medium,Increasing,Increasing,Expected to affect the city in the future,NaN,Serious,Low,NaN


### Combining Social Vulnerability & Hazards

In [77]:
combined = pd.merge(hazards, disadv_score, on=['City','State'], how='left')


### Creating Climate Hazard Variables

In [128]:
# higher number reflects higher degree of risk from climate change

hazard_score = hazards.copy()
hazard_score = hazard_score[(hazard_score['Consequence of hazard'] != 'Do not know') & (hazard_score['Probability of hazard'] != 'Do not know')]
hazard_score = hazard_score[hazard_score['Hazard status'] == 'Currently affecting the city']

timescale = {'Long-term':2.0, 'Medium-term':5.0,  'Short-term':10.0}
consequence = {'Low':2.0, 'Medium Low':4.0, 'Medium':6.0, 'Medium High':8.0, 'High':10.0}
magnitude = {'Less serious':2.0, 'Serious':5.0, 'Extremely serious':10.0}
probability = {'Low':0.1, 'Medium Low':0.4, 'Medium':0.6, 'Medium High':0.8, 'High':0.99}

hazard_score['Anticipated timescale'].replace(timescale, inplace=True)
hazard_score['Consequence of hazard'].replace(consequence, inplace=True)
hazard_score['Magnitude of impact'].replace(magnitude, inplace=True)
hazard_score['Probability of hazard'].replace(probability, inplace=True)

hazard_score['hazard'] = (hazard_score['Anticipated timescale']*hazard_score['Consequence of hazard']*hazard_score['Magnitude of impact']*hazard_score['Probability of hazard'])
hazard_score[hazard_score['hazard'].notnull()][['City','hazard']].sort_values('hazard')

Column Name,City,hazard
450,San Leandro,5.00
508,Encinitas,7.92
445,San Leandro,10.00
230,Knoxville,10.00
217,Anchorage,14.40
...,...,...
33,Houston,990.00
34,Houston,990.00
58,Denver,990.00
398,Somerville,990.00


## Analyzing City Survey Questions - Rough Work

### Section 2 - Hazards and Adaptation 

In [59]:
#2.0 Has a climate change risk or vulnerability assessment been undertaken for your local government area?
# looking at how many cities answered yes, no, in progress etc. 
cities_clean[(cities_clean['Question Number'] == '2.0')][['City','Response Answer']].groupby('Response Answer').count().sort_values('City',ascending=False)


,City
Response Answer,
Yes,244
No,103
In progress,82
Intending to undertake in future,36
Do not know,4


In [60]:
# 2.2a Please list the most significant climate hazards faced by your city and indicate the probability and consequence of these hazards, as well as the expected future change in frequency and intensity. 
# Please also select the top 3 assets or services that are affected by the climate hazard, and provide a description of the impact.

cities_clean_2_2a = cities_clean[cities_clean['Question Number'] == '2.2a']
cities_clean_2_2a = cities_clean_2_2a[cities_clean_2_2a['Response Answer'].notnull()]

cities_clean_2_2a.pivot_table(index=['Account Number', 'City', 'Row Number'],
                                     columns='Column Name', 
                                     values='Response Answer',
                                     aggfunc=lambda x: ' '.join(x)).reset_index()

Column Name,Account Number,City,Row Number,Anticipated timescale,Climate Hazards,Consequence of hazard,Future change in frequency,Future change in intensity,Hazard status,Impact description,Magnitude of impact,Probability of hazard,Top three assets/ services affected
0,1093,Atlanta,1,Medium-term,Drought,High,Increasing,Increasing,Expected to affect the city in the future,Droughts can affect the supply of potable wate...,Extremely serious,High,Food & agriculture Emergency management Water ...
1,1093,Atlanta,2,Short-term,Heat wave,High,Increasing,Increasing,Expected to affect the city in the future,Around 200 people die in the City of Atlanta a...,Extremely serious,High,Public health Emergency management Energy
2,1093,Atlanta,3,Short-term,Rain storm,High,Increasing,Increasing,Currently affecting the city,Rain storms causes floods in the city overload...,Extremely serious,High,Public health Water supply & sanitation Emerge...
3,1093,Atlanta,4,Long-term,Extreme hot days,Medium,Increasing,Increasing,Expected to affect the city in the future,Extreme hot days overload the electrical syste...,Serious,Medium,Emergency management Energy Public health
4,1093,Atlanta,5,Long-term,Tornado,Low,Increasing,Increasing,Expected to affect the city in the future,Tornadoes have been known to disrupt power lin...,Serious,Low,Energy Emergency management Information & comm...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1963,830945,Juruena,5,Short-term,Water-borne disease,Medium High,Decreasing,Decreasing,Currently affecting the city,NaN,Serious,Medium High,Water supply & sanitation Food & agriculture P...
1964,831812,Puerto Barrios,1,Short-term,Tropical storm,Medium,Decreasing,Decreasing,Currently affecting the city,NaN,Less serious,Medium,Water supply & sanitation Food & agriculture E...
1965,831834,Cioeste,1,Medium-term,Rain storm,Medium High,Increasing,Increasing,Expected to affect the city in the future,"Probabilidade de enchentes, deslizamentos de t...",Serious,Medium,Environment Residential Water supply & sanitation
1966,832009,Xalapa,1,Short-term,Flash/surface flood,Low,Increasing,None,Currently affecting the city,Las inundaciones que se producen son en ocasio...,Less serious,Low,Residential Law & order Public health


<span style="color:red">Great question to create some variables here, example could be combining low elevation coastal areas with cities that mentioned drought as a high probability occurance</span>

In [61]:
# 2.2a Please list the most significant climate hazards faced by your city and indicate the probability and consequence of these hazards, as well as the expected future change in frequency and intensity. 
# Please also select the top 3 assets or services that are affected by the climate hazard, and provide a description of the impact.

count_frame = cities_clean[(cities_clean['Question Number'] == '2.2a') & (cities_clean['Row Number'] == 1) & (cities_clean['Column Number'] == 1)].groupby('Response Answer').count()
count_frame[['City']].sort_values('City',ascending=False)

,City
Response Answer,
Rain storm,90
Heat wave,61
Drought,45
Flash/surface flood,40
Extreme hot days,33
River flood,26
Coastal flood,25
Cyclone (Hurricane/Typhoon),13
Severe wind,11


In [63]:
# 2.2a Please list the most significant climate hazards faced by your city and indicate the probability and consequence of these hazards, as well as the expected future change in frequency and intensity. 
# Please also select the top 3 assets or services that are affected by the climate hazard, and provide a description of the impact.

cities_clean[(cities_clean['Question Number'] == '2.2a') & (cities_clean['Column Number'] == 9)][['City','Response Answer']].groupby('Response Answer').count().sort_values('City',ascending=False)

,City
Response Answer,
Public health,633
Residential,563
Water supply & sanitation,494
Environment,480
Transport,480
Emergency management,401
Energy,334
Food & agriculture,299
Commercial,260


In [64]:
#2.4 Please identify the factors that most greatly affect your city’s ability to adapt to climate change, and indicate how those factors either enhance or challenge this ability.

cities_clean_2_4 = cities_clean[cities_clean['Question Number'] == '2.4']
cities_clean_2_4 = cities_clean_2_4[cities_clean_2_4['Response Answer'].notnull()]

cities_clean_2_4_wide = cities_clean_2_4.pivot_table(index=['Account Number', 'City', 'Row Number'],
                                     columns='Column Name', 
                                     values='Response Answer',
                                     aggfunc=lambda x: ' '.join(x)).reset_index()
cities_clean_2_4_wide

Column Name,Account Number,City,Row Number,Description,Enhance/ Challenge,Factors that affect ability to adapt
0,1093,Atlanta,1,Both the City and region have experienced sig...,Challenge,Rapid urbanization
1,1093,Atlanta,2,The City of Atlanta accounts for 59 percent of...,Challenge,Inequality
2,1093,Atlanta,3,Atlanta’s economy and housing market was hit ...,Challenge,Housing
3,1093,Atlanta,4,"In the past, Atlanta has normally experienced ...",Challenge,Environmental conditions
4,1093,Atlanta,5,"In 2014, the American Society of Civil Enginee...",Challenge,Infrastructure capacity
...,...,...,...,...,...,...
1096,831834,Cioeste,1,Obtenção de financiamento para a remoção de co...,Challenge,Inequality
1097,831834,Cioeste,2,"Melhorar a infra-estrutura de bairros, efetuar...",Challenge,Inequality
1098,831834,Cioeste,3,Criar na comunidade um senso de participação a...,Challenge,Inequality
1099,831834,Cioeste,4,Idem acima.,Challenge,Inequality


In [ ]:
#2.4 Please identify the factors that most greatly affect your city’s ability to adapt to climate change, and indicate how those factors either enhance or challenge this ability.

cities_clean_2_4_wide[cities_clean_2_4_wide['Enhance/ Challenge'] == 'Challenge'][['City','Factors that affect ability to adapt']].groupby('Factors that affect ability to adapt').count().sort_values('Organizat',ascending=False)

<span style="color:red">Similar to 2.2a, great question for variables, perhaps combining low income cities in comparison to country mean with ones that marked poverty or inequality as a challenge factor; also look into consumerism habits of people that are not in poverty; what level of income / spending promotes environmentalism vs consumerism</span>

### Section 4 - Social Risks

In [ ]:
# 4.0 Does your city face any social risks as a result of climate change?
cities_df[(cities_df['Question Number'] == '4.0') & (cities_df['Response Answer'] == 'Yes')][['Organization']].count()

In [ ]:
# 4.0a Please complete the table below to indicate which social risks your city faces as a result of climate change and indicate if these risks have been covered by your city’s action plan.
# looking at most common social risks
cities_df[(cities_df['Question Number'] == '4.0a') & (cities_df['Column Number'] == 1)][['Organization','Response Answer']].groupby('Response Answer').count().sort_values('Organization',ascending=False)

<span style="color:red">This could also be combined with a variety of other 'disadvantaged groups' variables</span>

### Opportunities

In [ ]:
# 5.0a Please indicate the opportunities and describe how the city is positioning itself to take advantage of them.
cities_df[(cities_df['Question Number'] == '5.0a') & (cities_df['Column Number'] == 1)][['Organization','Response Answer']].groupby('Response Answer').count().sort_values('Organization',ascending=False)

In [ ]:
# 5.1 Does your city collaborate in voluntary partnership with businesses in your city on sustainability projects?
cities_df[(cities_df['Question Number'] == '5.1') & (cities_df['Response Answer'] == 'Yes')][['Organization']].count()


In [ ]:
# 5.1a Please provide some key examples of how your city collaborates with business in the table below.
cities_df[(cities_df['Question Number'] == '5.1a') & (cities_df['Column Number'] == 1)][['Organization','Response Answer']].groupby('Response Answer').count().sort_values('Organization',ascending=False)


<span style="color:red">This could be interesting in matching up with corporations based on industry...</span>

In [ ]:
# 5.2 List any emission reduction, adaptation, water related or resilience projects you have planned within your city for which you hope to attract financing, and provide details on the
cities_df_5_2 = cities_df[cities_df['Question Number'] == '5.2']
cities_df_5_2 = cities_df_5_2[cities_df_5_2['Response Answer'].notnull()]

cities_df_5_2 = cities_df_5_2.pivot_table(index=['Account Number', 'Organization', 'Row Number'],
                                     columns='Column Name', 
                                     values='Response Answer',
                                     aggfunc=lambda x: ' '.join(x)).reset_index()
cities_df_5_2.head()

In [ ]:
# 5.2 List any emission reduction, adaptation, water related or resilience projects you have planned within your city for which you hope to attract financing, and provide details on the

cities_df[(cities_df['Question Number'] == '5.2') & (cities_df['Column Number'] == 1)][['Organization','Response Answer']].groupby('Response Answer').count().sort_values('Organization',ascending=False)

<span style="color:red">Something randomly thought of, could be interesting to match up projects from this question with hazards the city is facing as a way to generate project ideas?? Also projects partially funded match with corporations in that industry</span>

### Local Government Emissions

In [ ]:
#6.0 Do you have an emissions inventory for your local government operations to report?
cities_df[(cities_df['Question Number'] == '6.0') & (cities_df['Column Number'] == 0)][['Organization','Response Answer']].groupby('Response Answer').count().sort_values('Organization',ascending=False)
